<a href="https://colab.research.google.com/github/dviva1972/denvlaiva/blob/master/ML_RS_hw3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Иванов Денис

###  MovieLense:  рекомендательная система кинофильмов

**Краткое описание работы рекомендательной системы:**

**Rough selection**

Для рекомендательной системы выбраны только сведения о следующих фильмах:
- фильму проставлено хотя бы 10 оценок (исключаем редкие фильмы)
- медиана оценок фильма >=4 (исключаем фильмы рейтинга - ниже среднего допустимого / заботимся о репутации рейтинговой системы)
- оценки фильма соответствуют четвертому (лучшему) квартилю хотя бы 10 пользователей (если не так, то пользователи особых восторгов по фильму не испытывали и не стали бы его рекомендовать.)
- из состава анализируемых тэгов исключены словосочетания, встречающиеся менее 50 раз

**Poor selection**

По модели ALS (Юзер / Фильм / Рейтинг)  на всей выборке после Rough selection отобраны:
- по 100 фильмов наиболее похожих на последние 5 понравившихся фильмов рассматриваемого Пользователя (итого 500):
- 500 users, наиболее похожих на рассматриваемого Пользователя

На следующий этап отбора проходят только записи, относящиеся к полученной выборке.

**Model selection**

В отношении следующих баз данных:

- Юзер / Фильм / Рейтинг
- Тэг / Фильм / Медиана рейтинга по фильму
- Жанр / Фильм / Медиана рейтинга по фильму
(под Жанром понимаются каждый из признаков в составе movies.genres, а также агрегированные до 5+ лет сведения о годе выпуска фильма в составе movies.title)

по итогам применения каждой из следующих моделей Surprise:
- SlopeOne(), 
- SVD(), 
- BaselineOnly()
(или других каких нибудь сочетаний 2-3 моделей)

подобрано по 5 наиболее подходящих фильмов, при том что:
- самому подходящему (из каждых 5) присвоено 5 баллов 
- наименее подходящему  - 1 балл (между - пропорционально)

дополнительно по итогам работы модели ALS по 5 баллов присвоено фильмам:
- самому похожему фильму на каждый из 5 последних фильмов (понравившихся Пользователю)
- последний фильм с наивысшей оценкой для каждого из 5 наиболее похожих пользователей

В состав рекомендации попали 5 фильмов, набравших наибольшее количество баллов


In [1]:
import pandas as pd
import numpy as np
import re

from collections import defaultdict

from warnings import filterwarnings 
filterwarnings('ignore')

In [2]:
pip install surprise

     |████████████████████████████████| 11.8MB 14.9MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670909 sha256=e89a09774378cc2ac4e988c5d1ca3bec0ae10a236e370a68333e31e0aa520df4
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [7]:
from surprise import KNNBasic, SlopeOne, BaselineOnly, SVD
from surprise import Dataset
from surprise import Reader

In [8]:
pip install implicit

  Using cached https://files.pythonhosted.org/packages/bc/07/c0121884722d16e2c5beeb815f6b84b41cbf22e738e4075f1475be2791bc/implicit-0.4.4.tar.gz
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419392 sha256=7f1b4fb2222e7485f7ba9755ef5f6259898d628069c43a51656172dd45ccfd0f
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [9]:
import scipy.sparse as sparse
from implicit.als import AlternatingLeastSquares

In [10]:
import zipfile 
import os      
from google.colab import drive 

In [11]:
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
zip_file = '/content/drive/My Drive/ml-latest.zip'  

z = zipfile.ZipFile(zip_file, 'r')
z.extractall()
z.printdir()

File Name                                             Modified             Size
ml-latest/                                     2018-09-26 16:46:50            0
ml-latest/links.csv                            2018-09-26 16:37:10      1267039
ml-latest/tags.csv                             2018-09-26 16:11:24     39744990
ml-latest/genome-tags.csv                      2018-09-26 16:44:04        18103
ml-latest/ratings.csv                          2018-09-26 16:31:52    759200511
ml-latest/README.txt                           2018-09-26 16:46:50         9784
ml-latest/genome-scores.csv                    2018-09-26 16:44:04    414851573
ml-latest/movies.csv                           2018-09-26 16:35:42      2858223



Rough selection


In [13]:
movies  = pd.read_csv('ml-latest/movies.csv')
ratings = pd.read_csv('ml-latest/ratings.csv')
tags    = pd.read_csv('ml-latest/tags.csv')
ratings.head(1)

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221


In [14]:
ratings.shape, movies.shape, tags.shape

((27753444, 4), (58098, 3), (1108997, 4))

Из состава выборки рекомендуемых фильмов исключаем средние и плохие фильмы (медианный рейтинг меньше 4)

In [15]:
medi_ratings  = ratings.groupby(by='movieId').rating.median()
select_movies_1 = medi_ratings[medi_ratings >=4].index.tolist()
len(select_movies_1)

10202

Из состава выборки рекомендуемых фильмов исключаем редкие фильмы (менее  10 оценок)

In [16]:
select_movies_2 = ratings.movieId.value_counts()[ratings.movieId.value_counts()>10].index.tolist()
len(select_movies_2)

22856

В состав рекомендаций системы могут попасть фильмы, которые кому то уже (очень) понравились, то есть оценка за фильм должна находится в лучшем / четвертом квартиле оценок хотя бы одного пользователя. Если фильм никому не понравился - исключаем из выборки.

In [17]:
ratings = pd.merge(ratings,
                   ratings.groupby('userId').rating.quantile(0.75),
                   how = 'left',
                   left_on  = ['userId'],
                   right_on  = ['userId']) 

In [18]:
ratings.head(1)

,userId,movieId,rating_x,timestamp,rating_y
0,1,307,3.5,1256677221,4.125


In [19]:
ratings.shape

(27753444, 5)

In [20]:
ratings = ratings[ratings.rating_x > ratings.rating_y ]

In [21]:
select_movies_3 = ratings.movieId.unique().tolist()
len(select_movies_3)

27054

Исключаем из всех выборок фильмы по критериям, указанным выше

In [22]:
select_movies = set(select_movies_1 + select_movies_2 + select_movies_3)

In [23]:
ratings = ratings[ratings.movieId.isin(select_movies)]
movies  = movies[movies.movieId.isin(select_movies)]
tags    = tags[tags.movieId.isin(select_movies)]

ratings.shape, movies.shape, tags.shape

((3555325, 5), (31910, 3), (1038493, 4))

Feature selection

Movies

Представим себе, что каждый жанр, указанный в поле genres -  это тэг,  и сделаем:

In [24]:
genres_vect = movies["genres"].str.get_dummies("|").iloc[:,1:]

In [25]:
genres_vect.head(1)

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [26]:
i             = 0
col_s         = ['movieId','tag']
movie_tag     = pd.DataFrame(columns =col_s)


for i in range (19):
    df                      = pd.DataFrame(list(zip(movies.movieId, genres_vect.iloc[:,i])))
    df.columns              = col_s
    df.loc[df.tag>0, 'tag'] = genres_vect.columns[i]
    movie_tag               = pd.concat([movie_tag, df],axis = 0)

movie_tag = movie_tag.loc[movie_tag.tag != 0]

In [27]:
movie_tag.head(1)

,movieId,tag
5,6,Action


сформируем дополнительные признаки - год выпуска фильма и период выпуска фильма (с точностью до 5-10 лет)

In [28]:
def set_year(row):
    z = re.findall('(\d\d\d\d)', row.title)
    if len(z) == 0 or z == 0 or z == None:
        z = 9999
    else:
        z = int(z[-1])
    return z    

movies['year_of_film']= movies.apply(set_year, axis=1)

In [29]:
movies['tag'] = pd.cut(movies.year_of_film, [1900, 1960, 1970, 1980, 1990, 2000,
                                                       2005, 2010, 2015, 10000]) 
movies['tag'].value_counts()

(2010, 2015]     5991
(2005, 2010]     5081
(1990, 2000]     4830
(2000, 2005]     3574
(1900, 1960]     3021
(1980, 1990]     2945
(2015, 10000]    2666
(1970, 1980]     2094
(1960, 1970]     1685
Name: tag, dtype: int64

Соединим все синтетические тэги и оставим до лучших времен

In [30]:
movie_tag = pd.concat([movie_tag, 
                       movies.iloc[:,[0,4]]],
                       axis = 0)
movie_tag.tail()

,movieId,tag
58073,193835,"(2010.0, 2015.0]"
58077,193843,"(2015.0, 10000.0]"
58089,193868,"(1900.0, 1960.0]"
58091,193872,"(2015.0, 10000.0]"
58092,193874,"(1900.0, 1960.0]"


In [31]:
movie_tag.shape, movies.shape

((92463, 2), (31910, 5))

Tags

In [32]:
tags.head(1)

,userId,movieId,tag,timestamp
0,14,110,epic,1443148538


In [33]:
tags.tag.value_counts().head()

sci-fi         9381
atmospheric    6425
action         6136
comedy         5727
surreal        5262
Name: tag, dtype: int64

исключаем пустые записи

In [34]:
tags = tags[~tags.tag.isna()]

приводим все теги к нижнему регистру

In [35]:
tags['tag'] = tags['tag'].str.lower()

если наименование жанра содержится в теге - оставляем типовое поле о принадлежности тега к жанру, жанр уже отражен в базе movies тэгов

In [36]:
for i in genres_vect.columns.str.lower().tolist():
    tags.loc[tags.tag ==i, 'tag'] = 'genres_tag'

In [37]:
tags.tag.value_counts().head()

genres_tag         58751
atmospheric         6987
surreal             5534
funny               5527
based on a book     5338
Name: tag, dtype: int64

если время выпуска фильма содержится в теге - оставляем типовое поле о принадлежности тега ко времение, "эпоха" уже отражена в базе movies

In [38]:
tags[tags.tag.str.contains('0s')].head()

,userId,movieId,tag,timestamp
292,206,541,80s,1527550262
293,206,2115,80s,1527550065
295,206,2402,80s,1527652380
296,206,2403,80s,1527652469
297,206,3740,80s,1527550214


In [39]:
tags.loc[tags.tag.str.contains('0s'),  'tag'] = 'epoch_tag'
tags.loc[tags.tag.str.contains("0's"), 'tag'] = 'epoch_tag'

Исключим редкие тэги / встречаются менее 50 раз 

In [40]:
rare_tags = tags.tag.value_counts()[tags.tag.value_counts() < 50].index.tolist()
tags      = tags[~tags.tag.isin(rare_tags)]
tags.shape

(790192, 4)

Ratings

In [41]:
ratings.head(1)

,userId,movieId,rating_x,timestamp,rating_y
3,1,1257,4.5,1256677460,4.125


In [42]:
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit="s").dt.date
ratings.head(1)

,userId,movieId,rating_x,timestamp,rating_y
3,1,1257,4.5,2009-10-27,4.125


Для целей построения модели сформируем промещуточные базы :
- Тэг / Фильм / Рейтинг фильма (медиана)
- Жанр / Фильм / Рейтинг фильма (медиана)

In [43]:
base  = tags.merge(ratings.iloc[:,:3],
                   how      = 'left',
                   left_on  = ['userId','movieId'] ,
                   right_on = ['userId','movieId'])


In [44]:
base = base[~base.rating_x.isna()]
base.head(1)

,userId,movieId,tag,timestamp,rating_x
21,56,2028,disturbing,1274837927,4.0


In [45]:
base_2  = movie_tag.merge(  ratings.groupby(by = 'movieId').rating_x.median(),
                            how      = 'left',
                            left_on  = ['movieId'] ,
                            right_on = ['movieId'])
base_2.head(1)

,movieId,tag,rating_x
0,6,Action,5.0


In [46]:
base_2 = base_2[base_2.rating_x>0]

In [47]:
base_2.head(1)

,movieId,tag,rating_x
0,6,Action,5.0


Poor selection

In [48]:
users_list   = list(np.sort(ratings.userId))
movies_list  = list(np.sort(ratings.movieId))
rating_list  = list(ratings.rating_x)

In [49]:
cols = ratings.movieId.astype(int)
rows = ratings.userId.astype(int)

In [50]:
data_sparse = sparse.csr_matrix((rating_list, 
                                (cols, rows)), 
                                shape=(len(users_list), len(movies_list)))

In [51]:
model = AlternatingLeastSquares(factors=50)
model.fit(data_sparse)

GPU training requires factor size to be a multiple of 32. Increasing factors from 50 to 64.


Модель рекомендательной системы

In [64]:
list_alg = [SlopeOne(), 
            KNNBasic(sim_options= {'name': 'cosine'}),
            BaselineOnly(bsl_options={'method': 'als', 'n_epochs': 5 }) 
            ]
list_alg_2=[SlopeOne(), 
            SVD(),
            BaselineOnly(bsl_options={'method': 'als', 'n_epochs': 5 }) 
            ]
list_alg_3=[SlopeOne(), 
            KNNBasic(sim_options= {'name': 'cosine'})
            ]

In [65]:
def get_top_n(predictions, n=3):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [66]:
def find_films (current_user, algorithms=list_alg,  count_predict= 5):

    cu_last_films  = ratings[ratings.userId == current_user].sort_values(
                    'timestamp', ascending = False).movieId[:5].tolist()

    select_films   = pd.DataFrame(columns = ['film', 'rate'])

    related_movies        = []
    select_related_movies = []
    sim_users             = []

    for u in cu_last_films:
        select_related_movies.append(model.similar_items(u,100)[1][0])
        for i in model.similar_items(u,101)[1:]:
            related_movies.append(i[0])

    for i in model.similar_users(current_user,501)[1:]:
        sim_users.append(i[0])

    for i in sim_users[:5]:
        select_related_movies.append(ratings.sort_values(
                                 ['rating_x','timestamp'], ascending = False).
                                 loc[ratings.userId ==i, 'movieId'].values[0])
    
    ratings_for_model = ratings[(ratings.movieId.isin(related_movies))&
                                (ratings.userId.isin( sim_users))]
    base_for_model   = base  [(base.movieId.isin(  related_movies))]
    base_2_for_model = base_2[(base_2.movieId.isin(related_movies))]

    dataset_1 = pd.DataFrame({
                            'uid':    ratings_for_model.userId,
                            'iid':    ratings_for_model.movieId,
                            'rating': ratings_for_model.rating_x #individual_rate
                            })

    dataset_2 = pd.DataFrame({
                            'uid':    base_for_model.tag,
                            'iid':    base_for_model.movieId,
                            'rating': base_for_model.rating_x  # median rate
                            })

    dataset_3 = pd.DataFrame({
                            'uid':    base_2_for_model.tag,
                            'iid':    base_2_for_model.movieId,
                            'rating': base_2_for_model.rating_x # median  rate
                            })

    datasets = [dataset_1, dataset_2, dataset_3]
    
    params       = [sim_users[:5],  cu_last_films, cu_last_films]
    select_films = pd.DataFrame(list(zip(select_related_movies, 
                                         [5]*len(select_related_movies))),
                                columns =['movieId', 'rate'])

    for ds in datasets:
        reader      = Reader(line_format='user item rating', 
                             rating_scale=(0.5,5.0))
        df_prepared = Dataset.load_from_df(ds, reader)
        trainset    = df_prepared.build_full_trainset()
        testset     =  trainset.build_anti_testset()
        p=0
   
        for la in algorithms:
            algo   = la
            algo.fit (trainset)
            predictions = algo.test(testset)
            top_n = get_top_n(predictions, n=count_predict)
            for us in params[p]:
                r = count_predict
                for i in top_n[us]:
                    select_films = select_films.append(
                        [{'movieId':i[0], 'rate':r}], ignore_index=True)
                    r  = r-1
            p +=1

    select_films = select_films[~select_films.movieId.isin(cu_last_films)]
    top_films    = select_films.groupby(by='movieId'
                            ).rate.sum().sort_values(ascending=False)[:5]

    print ('Список рекомендованных фильмов')
    for i in  top_films.index.tolist():
        print (movies[movies.movieId == i].iloc[:,1].min())
    
    print ('Список последних понравившихся фильмов / для сравнения')
    for i in  cu_last_films:
        print (movies[movies.movieId == i].iloc[:,1].min())  
    


In [67]:
find_films(68040) #SlopeOne + KNNBasic +  BaselineOnly

Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Список рекомендованных фильмов
Clerks (1994)
Mallrats (1995)
All About Eve (1950)
Gallipoli (1981)
48 Hrs. (1982)
Список последних понравившихся фильмов / для сравнения
Office Space (1999)
Rope (1948)
All That Heaven Allows (1955)
Memento (2000)
Beverly Hills Cop (1984)


In [68]:
find_films(68040, algorithms=list_alg_2) #SlopeOne + SVD +  BaselineOnly

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Список рекомендованных фильмов
Clerks (1994)
Mallrats (1995)
All About Eve (1950)
Gallipoli (1981)
48 Hrs. (1982)
Список последних понравившихся фильмов / для сравнения
Office Space (1999)
Rope (1948)
All That Heaven Allows (1955)
Memento (2000)
Beverly Hills Cop (1984)


In [69]:
find_films(68040, algorithms=list_alg_3) #SlopeOne + KNNBasic

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Список рекомендованных фильмов
Clerks (1994)
Mallrats (1995)
All About Eve (1950)
Gallipoli (1981)
48 Hrs. (1982)
Список последних понравившихся фильмов / для сравнения
Office Space (1999)
Rope (1948)
All That Heaven Allows (1955)
Memento (2000)
Beverly Hills Cop (1984)


In [70]:
find_films(123659)                       #SlopeOne + KNNBasic +  BaselineOnly

Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Список рекомендованных фильмов
Weird Science (1985)
Better Off Dead... (1985)
Do the Right Thing (1989)
This Is Spinal Tap (1984)
Monty Python Live at the Hollywood Bowl (1982)
Список последних понравившихся фильмов / для сравнения
Bottle Rocket (1996)
Defending Your Life (1991)
Fletch (1985)
Rushmore (1998)
Monty Python's And Now for Something Completely Different (1971)


In [71]:
find_films(123659, algorithms=list_alg_2)   #SlopeOne + SVD + BaselineOnly

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Список рекомендованных фильмов
Weird Science (1985)
Better Off Dead... (1985)
Do the Right Thing (1989)
This Is Spinal Tap (1984)
Monty Python Live at the Hollywood Bowl (1982)
Список последних понравившихся фильмов / для сравнения
Bottle Rocket (1996)
Defending Your Life (1991)
Fletch (1985)
Rushmore (1998)
Monty Python's And Now for Something Completely Different (1971)


In [72]:
find_films(123659, algorithms=list_alg_3)   #SlopeOne + KNNBasic +  BaselineOnly

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Список рекомендованных фильмов
Weird Science (1985)
Better Off Dead... (1985)
Do the Right Thing (1989)
This Is Spinal Tap (1984)
Monty Python Live at the Hollywood Bowl (1982)
Список последних понравившихся фильмов / для сравнения
Bottle Rocket (1996)
Defending Your Life (1991)
Fletch (1985)
Rushmore (1998)
Monty Python's And Now for Something Completely Different (1971)


In [73]:
find_films(1111)                            #SlopeOne + KNNBasic +  BaselineOnly

Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Список рекомендованных фильмов
Heathers (1989)
Killing Fields, The (1984)
Waiting for Guffman (1996)
Weird Science (1985)
Do the Right Thing (1989)
Список последних понравившихся фильмов / для сравнения
Cinema Paradiso (Nuovo cinema Paradiso) (1989)
Princess Bride, The (1987)
Better Off Dead... (1985)
This Is Spinal Tap (1984)
Real Genius (1985)


In [74]:
find_films(1111, algorithms=list_alg_2)     #SlopeOne + SVD + BaselineOnly

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Список рекомендованных фильмов
Heathers (1989)
Killing Fields, The (1984)
Waiting for Guffman (1996)
Weird Science (1985)
Do the Right Thing (1989)
Список последних понравившихся фильмов / для сравнения
Cinema Paradiso (Nuovo cinema Paradiso) (1989)
Princess Bride, The (1987)
Better Off Dead... (1985)
This Is Spinal Tap (1984)
Real Genius (1985)


In [75]:
find_films(1111, algorithms=list_alg_3)     #SlopeOne + KNNBasic

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Список рекомендованных фильмов
Heathers (1989)
Killing Fields, The (1984)
Waiting for Guffman (1996)
Weird Science (1985)
Do the Right Thing (1989)
Список последних понравившихся фильмов / для сравнения
Cinema Paradiso (Nuovo cinema Paradiso) (1989)
Princess Bride, The (1987)
Better Off Dead... (1985)
This Is Spinal Tap (1984)
Real Genius (1985)


.